In [0]:
!git clone https://github.com/fizyr/keras-retinanet.git
%cd /content/keras-retinanet/
!pip install .
%cd /content/keras-retinanet/
!python setup.py build_ext --inplace
%cd /content/

In [0]:
from google.colab import drive
drive.mount('/content/drive')

###Imports

In [0]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import keras
import keras.backend as K
from keras.applications import resnet50
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications import inception_v3
from keras.utils import plot_model

# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color
from keras_retinanet.utils.gpu import setup_gpu
from keras_retinanet.utils.compute_overlap import compute_overlap

# import miscellaneous modules
import matplotlib.pyplot as plt
import cv2
import os
import numpy as np
import scipy
import time
import argparse
from __future__ import print_function
import urllib
from google.colab import files
from google.colab import drive
import time
from datetime import datetime

# cv2.imshow() causes jupyter colapse
from google.colab.patches import cv2_imshow

# import tensorflow v1
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

# use this to change which GPU to use
gpu = 0

# set the modified tf session as backend in keras
setup_gpu(gpu)

drive_path = '/content/drive/' # set to your desired folder
save_path = drive_path         # set to your desired folder

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

###Model

In [5]:
### import model
model_url = 'https://github.com/fizyr/keras-retinanet/releases/download/0.5.1/resnet50_coco_best_v2.1.0.h5'
model_path = os.path.join('.', 'keras-retinanet', 'snapshots', 'resnet50_coco.h5')
urllib.request.urlretrieve(model_url, model_path)

# We will not be training our model,
# so we use this command to disable all training-specific operations
K.set_learning_phase(0)

# load retinanet model
model = models.load_model(model_path, backbone_name='resnet50')

# load label to names mapping for visualization purposes
labels_to_names = {0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table', 61: 'toilet', 62: 'tv', 63: 'laptop', 64: 'mouse', 65: 'remote', 66: 'keyboard', 67: 'cell phone', 68: 'microwave', 69: 'oven', 70: 'toaster', 71: 'sink', 72: 'refrigerator', 73: 'book', 74: 'clock', 75: 'vase', 76: 'scissors', 77: 'teddy bear', 78: 'hair drier', 79: 'toothbrush'}

model_layers_dict = dict([(layer.name, layer) for layer in model.layers])

fig_size = (10,10)
mode = 'caffe'

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
tracking <tf.Variable 'Variable:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_1:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_2:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_3:0' shape=(9, 4) dtype=float32> anchors
tracking <tf.Variable 'Variable_4:0' shape=(9, 4) dtype=float32> anchors


/usr/local/lib/python3.6/dist-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


### defs: focal loss and compute targets

In [0]:
batch_size=1
num_classes = 80

def focal(alpha=0.25, gamma=2.0):
    """ Create a functor for computing the focal loss.

    Args
        alpha: Scale the focal weight with alpha.
        gamma: Take the power of the focal weight with gamma.

    Returns
        A functor that computes the focal loss using the alpha and gamma.
    """
    def _focal(y_true, y_pred):
        """ Compute the focal loss given the target tensor and the predicted tensor.

        As defined in https://arxiv.org/abs/1708.02002

        Args
            y_true: Tensor of target data from the generator with shape (B, N, num_classes).
            y_pred: Tensor of predicted data from the network with shape (B, N, num_classes).

        Returns
            The focal loss of y_pred w.r.t. y_true.
        """
        labels         = y_true[:, :, :-1]
        anchor_state   = y_true[:, :, -1]  # -1 for ignore, 0 for background, 1 for object
        classification  = y_pred

        # filter out "ignore" anchors
        indices        = tf.where(keras.backend.not_equal(anchor_state, -1))
        labels         = tf.gather_nd(labels, indices)
        classification  = tf.gather_nd(classification, indices)

        # compute the focal loss
        alpha_factor = keras.backend.ones_like(labels) * alpha
        alpha_factor = tf.where(keras.backend.equal(labels, 1), alpha_factor, 1 - alpha_factor)
        focal_weight = tf.where(keras.backend.equal(labels, 1), 1 - classification, classification)
        focal_weight = alpha_factor * focal_weight ** gamma

        cls_loss = focal_weight * keras.backend.binary_crossentropy(labels, classification)

        # compute the normalizer: the number of positive anchors
        normalizer = tf.where(keras.backend.equal(anchor_state, 1))
        normalizer = keras.backend.cast(keras.backend.shape(normalizer)[0], keras.backend.floatx())
        normalizer = keras.backend.maximum(keras.backend.cast_to_floatx(1.0), normalizer)

        return keras.backend.sum(cls_loss) / normalizer

    return _focal


def smooth_l1(sigma=3.0):
    """ Create a smooth L1 loss functor.

    Args
        sigma: This argument defines the point where the loss changes from L2 to L1.

    Returns
        A functor for computing the smooth L1 loss given target data and predicted data.
    """
    sigma_squared = sigma ** 2

    def _smooth_l1(y_true, y_pred):
        """ Compute the smooth L1 loss of y_pred w.r.t. y_true.

        Args
            y_true: Tensor from the generator of shape (B, N, 5). The last value for each box is the state of the anchor (ignore, negative, positive).
            y_pred: Tensor from the network of shape (B, N, 4).

        Returns
            The smooth L1 loss of y_pred w.r.t. y_true.
        """
        # separate target and state
        regression        = y_pred
        regression_target = y_true[:, :, :-1]
        anchor_state      = y_true[:, :, -1]

        # filter out "ignore" anchors
        indices           = tf.where(keras.backend.equal(anchor_state, 1))
        regression        = tf.gather_nd(regression, indices)
        regression_target = tf.gather_nd(regression_target, indices)

        # compute smooth L1 loss
        # f(x) = 0.5 * (sigma * x)^2          if |x| < 1 / sigma / sigma
        #        |x| - 0.5 / sigma / sigma    otherwise
        regression_diff = regression - regression_target
        regression_diff = keras.backend.abs(regression_diff)
        regression_loss = tf.where(
            keras.backend.less(regression_diff, 1.0 / sigma_squared),
            0.5 * sigma_squared * keras.backend.pow(regression_diff, 2),
            regression_diff - 0.5 / sigma_squared
        )

        # compute the normalizer: the number of positive anchors
        normalizer = keras.backend.maximum(1, keras.backend.shape(indices)[0])
        normalizer = keras.backend.cast(normalizer, dtype=keras.backend.floatx())
        return keras.backend.sum(regression_loss) / normalizer

    return _smooth_l1

def preprocess_group_entry(image, annotations):
    """ Preprocess image and its annotations.
    """
    # preprocess the image
    image = preprocess_image(image)

    # convert to the wanted keras floatx
    image = K.cast_to_floatx(image)

    return image, annotations

def preprocess_group(image_group, annotations_group):
    """ Preprocess each image and its annotations in its group.
    """
    assert(len(image_group) == len(annotations_group))

    for index in range(len(image_group)):
        # preprocess a single group entry
        image_group[index], annotations_group[index] = preprocess_group_entry(image_group[index], annotations_group[index])

    return image_group, annotations_group

def compute_inputs(image_group):
    """ Compute inputs for the network using an image_group.
    """
    # get the max image shape
    max_shape = tuple(max(image.shape[x] for image in image_group) for x in range(3))

    # construct an image batch object
    image_batch = np.zeros((batch_size,) + max_shape, dtype=K.floatx())

    # copy all images to the upper left part of the image batch object
    for image_index, image in enumerate(image_group):
        image_batch[image_index, :image.shape[0], :image.shape[1], :image.shape[2]] = image

    if K.image_data_format() == 'channels_first':
        image_batch = image_batch.transpose((0, 3, 1, 2))

    return image_batch

def compute_targets(image_group, annotations_group):
    """ Compute target outputs for the network using images and their annotations.
    """
    # get the max image shape
    max_shape = tuple(max(image.shape[x] for image in image_group) for x in range(3))
    anchors   = anchors_for_shape(max_shape)

    batches = anchor_targets_bbox(
        anchors,
        image_group,
        annotations_group,
        num_classes
    )

    return list(batches)

class AnchorParameters:
    """ The parameteres that define how anchors are generated.

    Args
        sizes   : List of sizes to use. Each size corresponds to one feature level.
        strides : List of strides to use. Each stride correspond to one feature level.
        ratios  : List of ratios to use per location in a feature map.
        scales  : List of scales to use per location in a feature map.
    """
    def __init__(self, sizes, strides, ratios, scales):
        self.sizes   = sizes
        self.strides = strides
        self.ratios  = ratios
        self.scales  = scales

    def num_anchors(self):
        return len(self.ratios) * len(self.scales)
"""
The default anchor parameters.
"""
AnchorParameters.default = AnchorParameters(
    sizes   = [32, 64, 128, 256, 512],
    strides = [8, 16, 32, 64, 128],
    ratios  = np.array([0.5, 1, 2], keras.backend.floatx()),
    scales  = np.array([2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)], keras.backend.floatx()),
)

def anchors_for_shape(
    image_shape,
    pyramid_levels=None,
    anchor_params=None,
    shapes_callback=None,
):
    """ Generators anchors for a given shape.

    Args
        image_shape: The shape of the image.
        pyramid_levels: List of ints representing which pyramids to use (defaults to [3, 4, 5, 6, 7]).
        anchor_params: Struct containing anchor parameters. If None, default values are used.
        shapes_callback: Function to call for getting the shape of the image at different pyramid levels.

    Returns
        np.array of shape (N, 4) containing the (x1, y1, x2, y2) coordinates for the anchors.
    """

    if pyramid_levels is None:
        pyramid_levels = [3, 4, 5, 6, 7]

    if anchor_params is None:
        anchor_params = AnchorParameters.default

    if shapes_callback is None:
        shapes_callback = guess_shapes
    image_shapes = shapes_callback(image_shape, pyramid_levels)
    print('Image shapes:', image_shapes)

    # compute anchors over all pyramid levels
    all_anchors = np.zeros((0, 4))
    for idx, p in enumerate(pyramid_levels):
        anchors = generate_anchor_windows(                                              
            base_size=anchor_params.sizes[idx],
            ratios=anchor_params.ratios,
            scales=anchor_params.scales
        )
        shifted_anchors = shift(image_shapes[idx], anchor_params.strides[idx], anchors)  
        all_anchors     = np.append(all_anchors, shifted_anchors, axis=0)

    return all_anchors

def generate_anchor_windows(base_size=16, ratios=None, scales=None): 
    """
    Generate anchor (reference) windows by enumerating aspect ratios X
    scales w.r.t. a reference window.
    """

    if ratios is None:
        ratios = AnchorParameters.default.ratios

    if scales is None:
        scales = AnchorParameters.default.scales

    num_anchors = len(ratios) * len(scales)

    # initialize output anchors
    anchors = np.zeros((num_anchors, 4))

    # scale base_size
    anchors[:, 2:] = base_size * np.tile(scales, (2, len(ratios))).T

    # compute areas of anchors
    areas = anchors[:, 2] * anchors[:, 3]

    # correct for ratios
    anchors[:, 2] = np.sqrt(areas / np.repeat(ratios, len(scales)))
    anchors[:, 3] = anchors[:, 2] * np.repeat(ratios, len(scales))

    # transform from (x_ctr, y_ctr, w, h) -> (x1, y1, x2, y2)
    anchors[:, 0::2] -= np.tile(anchors[:, 2] * 0.5, (2, 1)).T
    anchors[:, 1::2] -= np.tile(anchors[:, 3] * 0.5, (2, 1)).T

    return anchors

def shift(shape, stride, anchors):
    """ Produce shifted anchors based on shape of the map and stride size.

    Args
        shape  : Shape to shift the anchors over.
        stride : Stride to shift the anchors with over the shape.
        anchors: The anchors to apply at each location.
    """

    # create a grid starting from half stride from the top left corner
    shift_x = (np.arange(0, shape[1]) + 0.5) * stride
    shift_y = (np.arange(0, shape[0]) + 0.5) * stride

    shift_x, shift_y = np.meshgrid(shift_x, shift_y)

    shifts = np.vstack((
        shift_x.ravel(), shift_y.ravel(),
        shift_x.ravel(), shift_y.ravel()
    )).transpose()

    # add A anchors (1, A, 4) to
    # cell K shifts (K, 1, 4) to get
    # shift anchors (K, A, 4)
    # reshape to (K*A, 4) shifted anchors
    A = anchors.shape[0]
    K = shifts.shape[0]
    all_anchors = (anchors.reshape((1, A, 4)) + shifts.reshape((1, K, 4)).transpose((1, 0, 2)))
    all_anchors = all_anchors.reshape((K * A, 4))

    return all_anchors


def guess_shapes(image_shape, pyramid_levels):
    """Guess shapes based on pyramid levels.

    Args
         image_shape: The shape of the image.
         pyramid_levels: A list of what pyramid levels are used.

    Returns
        A list of image shapes at each pyramid level.
    """
    image_shape = np.array(image_shape[:2])
    image_shapes = [(image_shape + 2 ** x - 1) // (2 ** x) for x in pyramid_levels]
    return image_shapes



def anchor_targets_bbox(
    anchors,
    image_group,
    annotations_group,
    num_classes,
    negative_overlap=0.4,
    positive_overlap=0.5
):
    """ Generate anchor targets for bbox detection.

    Args
        anchors: np.array of annotations of shape (N, 4) for (x1, y1, x2, y2).
        image_group: List of BGR images.
        annotations_group: List of annotations (np.array of shape (N, 5) for (x1, y1, x2, y2, label)).
        num_classes: Number of classes to predict.
        mask_shape: If the image is padded with zeros, mask_shape can be used to mark the relevant part of the image.
        negative_overlap: IoU overlap for negative anchors (all anchors with overlap < negative_overlap are negative).
        positive_overlap: IoU overlap or positive anchors (all anchors with overlap > positive_overlap are positive).

    Returns
        labels_batch: batch that contains labels & anchor states (np.array of shape (batch_size, N, num_classes + 1),
                      where N is the number of anchors for an image and the last column defines the anchor state (-1 for ignore, 0 for bg, 1 for fg).
        regression_batch: batch that contains bounding-box regression targets for an image & anchor states (np.array of shape (batch_size, N, 4 + 1),
                      where N is the number of anchors for an image, the first 4 columns define regression targets for (x1, y1, x2, y2) and the
                      last column defines anchor states (-1 for ignore, 0 for bg, 1 for fg).
    """

    assert(len(image_group) == len(annotations_group)), "The length of the images and annotations need to be equal."
    assert(len(annotations_group) > 0), "No data received to compute anchor targets for."
    for annotations in annotations_group:
        assert('bboxes' in annotations), "Annotations should contain bboxes."
        assert('labels' in annotations), "Annotations should contain labels."

    batch_size = len(image_group)

    regression_batch  = np.zeros((batch_size, anchors.shape[0], 4 + 1), dtype=K.floatx())
    labels_batch      = np.zeros((batch_size, anchors.shape[0], num_classes + 1), dtype=K.floatx())

    # compute labels and regression targets
    for index, (image, annotations) in enumerate(zip(image_group, annotations_group)):
        if annotations['bboxes'].shape[0]:
            # obtain indices of gt annotations with the greatest overlap
            positive_indices, ignore_indices, argmax_overlaps_inds = compute_gt_annotations(anchors, annotations['bboxes'], negative_overlap, positive_overlap)

            labels_batch[index, ignore_indices, -1]       = -1
            labels_batch[index, positive_indices, -1]     = 1

            regression_batch[index, ignore_indices, -1]   = -1
            regression_batch[index, positive_indices, -1] = 1

            # compute target class labels
            labels_batch[index, positive_indices, annotations['labels'][argmax_overlaps_inds[positive_indices]].astype(int)] = 1

            regression_batch[index, :, :-1] = bbox_transform(anchors, annotations['bboxes'][argmax_overlaps_inds, :])

        # ignore annotations outside of image
        if image.shape:
            anchors_centers = np.vstack([(anchors[:, 0] + anchors[:, 2]) / 2, (anchors[:, 1] + anchors[:, 3]) / 2]).T
            indices = np.logical_or(anchors_centers[:, 0] >= image.shape[1], anchors_centers[:, 1] >= image.shape[0])

            labels_batch[index, indices, -1]     = -1
            regression_batch[index, indices, -1] = -1

    return regression_batch, labels_batch

def compute_gt_annotations(
    anchors,
    annotations,
    negative_overlap=0.4,
    positive_overlap=0.5
):
    """ Obtain indices of gt annotations with the greatest overlap.

    Args
        anchors: np.array of annotations of shape (N, 4) for (x1, y1, x2, y2).
        annotations: np.array of shape (N, 5) for (x1, y1, x2, y2, label).
        negative_overlap: IoU overlap for negative anchors (all anchors with overlap < negative_overlap are negative).
        positive_overlap: IoU overlap or positive anchors (all anchors with overlap > positive_overlap are positive).

    Returns
        positive_indices: indices of positive anchors
        ignore_indices: indices of ignored anchors
        argmax_overlaps_inds: ordered overlaps indices
    """

    overlaps = compute_overlap(anchors.astype(np.float64), annotations.astype(np.float64))   ### from keras_retinanet.utils.compute_overlap import compute_overlap
    argmax_overlaps_inds = np.argmax(overlaps, axis=1)
    max_overlaps = overlaps[np.arange(overlaps.shape[0]), argmax_overlaps_inds]

    # assign "dont care" labels
    positive_indices = max_overlaps >= positive_overlap
    ignore_indices = (max_overlaps > negative_overlap) & ~positive_indices

    return positive_indices, ignore_indices, argmax_overlaps_inds

def bbox_transform(anchors, gt_boxes, mean=None, std=None):
    """Compute bounding-box regression targets for an image."""

    if mean is None:
        mean = np.array([0, 0, 0, 0])
    if std is None:
        std = np.array([0.2, 0.2, 0.2, 0.2])

    if isinstance(mean, (list, tuple)):
        mean = np.array(mean)
    elif not isinstance(mean, np.ndarray):
        raise ValueError('Expected mean to be a np.ndarray, list or tuple. Received: {}'.format(type(mean)))

    if isinstance(std, (list, tuple)):
        std = np.array(std)
    elif not isinstance(std, np.ndarray):
        raise ValueError('Expected std to be a np.ndarray, list or tuple. Received: {}'.format(type(std)))

    anchor_widths  = anchors[:, 2] - anchors[:, 0]
    anchor_heights = anchors[:, 3] - anchors[:, 1]

    targets_dx1 = (gt_boxes[:, 0] - anchors[:, 0]) / anchor_widths
    targets_dy1 = (gt_boxes[:, 1] - anchors[:, 1]) / anchor_heights
    targets_dx2 = (gt_boxes[:, 2] - anchors[:, 2]) / anchor_widths
    targets_dy2 = (gt_boxes[:, 3] - anchors[:, 3]) / anchor_heights

    targets = np.stack((targets_dx1, targets_dy1, targets_dx2, targets_dy2))
    targets = targets.T

    targets = (targets - mean) / std

    return targets


def compute_input_output(image_group, annotations_group):
    """ Compute inputs and target outputs for the network.
    """
    # perform preprocessing steps
    image_group, annotations_group = preprocess_group(image_group, annotations_group)

    # compute network inputs
    inputs = compute_inputs(image_group)

    # compute network targets
    targets = compute_targets(image_group, annotations_group)

    return inputs, targets

###defs: preprocess, run dream, detection

In [0]:
# preprocess_image, read_image_bgr, resize_image imported from keras_retinanet.utils.image

def load_and_resize_image(path, min_side=800, max_side=1333):
  in_img = read_image_bgr(path)
  in_img, scale = resize_image(in_img, min_side=min_side, max_side=max_side)
  return in_img, scale

def deprocess_image(p_img, clip=True, touint8=True):
  img = np.copy(p_img)
  if mode == 'tf':
    img += 1.
    img *= 127.5
  
  elif mode == 'caffe':
    imagenet_mean = np.array([103.939, 116.779, 123.68])
    img += imagenet_mean
  
  if clip:
    img = np.clip(img, 0, 255)

  if touint8:
    return np.uint8(img)
  else:
    return img

def eval_loss_and_grads(x, fetch_loss_and_grads):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

def draw_caption_my(image, box, caption, color):
    """ Draws a caption above the box in an image.

    # Arguments
        image   : The image to draw on.
        box     : A list of 4 elements (x1, y1, x2, y2).
        caption : String containing the text to draw.
    """
    font = cv2.FONT_HERSHEY_PLAIN
    font_scale = 1

    (text_width, text_height) = cv2.getTextSize(caption, font, fontScale=font_scale, thickness=2)[0]
    b = np.array(box).astype(int)
    text_offset_x = b[0] + 1
    text_offset_y = b[1] + text_height + 3
    box_coords = ((text_offset_x-2, text_offset_y+2), (text_offset_x + text_width, text_offset_y - text_height - 3))
    cv2.rectangle(image, box_coords[0], box_coords[1], color, cv2.FILLED)
    cv2.putText(image, caption, (text_offset_x, text_offset_y), font, font_scale, (0, 0, 0), 2)
    cv2.putText(image, caption, (text_offset_x, text_offset_y), font, font_scale, (255, 255, 255), 1)



# detection on BGR image
def detect(image, save=None, min_score=0.5, anns=None):
  print("DETECTION, anns:", anns)
  
  # copy to RGB draw on
  draw = image.copy()
  draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

  # preprocess BGR image for network
  image = preprocess_image(image)

  # process image
  boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))

  # draw original bb (white)
  if anns is not None:
    b = anns['bboxes'][0].astype(int)
    cv2.rectangle(draw, (b[0], b[1]), (b[2], b[3]), (255,255,255), 1, cv2.LINE_AA)
    draw_caption_my(draw, b, labels_to_names[anns['labels'][0]], (255,255,255))

  # visualize detections
  for box, score, label in reversed(list(zip(boxes[0], scores[0], labels[0]))):
      # scores are sorted
      if score < min_score:
          continue
          
      color = label_color(label)
      # draw box
      b = box.astype(int)
      draw_box(draw, b, color=color)
      # draw caption
      caption = "{} {:.3f}".format(labels_to_names[label], score)
      draw_caption_my(draw, b, caption, color)
      
  cv2_imshow(draw[:,:,::-1]) # covert draw from RGB to BGR

  if save is not None:
    cv2.imwrite('{}-detections.png'.format(save), draw[:,:,::-1])

  return boxes[0], scores[0], labels[0]



def dream_and_save(in_img, model, params, layer_dict=None,
                   show=True, save=True, 
                   use_octaves=False):
  
  out_img_batch, out_loss, out_process_time, out_d_images = deep_dream(in_img, model, params, layer_dict=layer_dict)
    
  dtnow = datetime.now()
  timestamp = dtnow.strftime("%Y-%m-%d_%H-%M-%S")
  
  if params['layer_name'] is None:
    result_prefix = 'RN_anns' + '_eta-' + str(params['eta']) + '_its-' + str(params['iterations']) + '_mode-' + mode
  else:
    result_prefix = 'RN_lr-' + params['layer_name'] + '_ch-' + str(params['channel_index']) + '_eta-' + str(params['eta']) + '_its-' + str(params['iterations']) + '_mode-' + mode

  dep_img = deprocess_image(out_img_batch[0])

  # save clean image (cv2 takes BGR)
  cv2.imwrite('{}{}_{}_loss-{:.2f}_time-{:.2f}-clean.png'.format(save_path, timestamp, result_prefix, out_loss, out_process_time), dep_img)
  
  # plot and save plot image
  plt.figure(figsize=fig_size)
  print(fig_size)
  plt.axis('off')
  plt.imshow(dep_img[:,:,::-1])     # to RGB pre zobrazenie
  if params['layer_name'] is not None:
    plt.title('{}\nloss: {:.2f} time: {:.2f}'.format(result_prefix, out_loss, out_process_time))
  else:
    plt.title('{}\nL-BFGS: {} loss: {:.2f} time: {:.2f}\nlabels:{} bboxes:{}'.format(result_prefix, params['lbfgs'], out_loss, out_process_time, params['ann']['labels'], params['ann']['bboxes']))
  plt.tight_layout()
  if save:
    plt.savefig('{}{}_{}_loss-{:.2f}_time-{:.2f}.png'.format(save_path, timestamp, result_prefix, out_loss, out_process_time))
  if show:
    plt.show()
  
  return dep_img, out_img_batch[0], '{}{}_{}_loss-{:.2f}_time-{:.2f}'.format(save_path, timestamp, result_prefix, out_loss, out_process_time), out_d_images

# script to run multiple deams
def run_dream(img, model, settings, show=True, save=True):
  model_input = model.input
  layer_dict = dict([(layer.name, layer) for layer in model.layers])
  dreams = []

  for i, layer_name in enumerate(settings['layers']):
    for channel_index in settings['ch_indexes']:
      if layer_name is None:
        print('PROCESSING anns: {}'.format(settings['ann']))
      else:
        print('PROCESSING layer: {} channel: {}'.format(layer_name, channel_index))
      params = {
          'layer_name': layer_name,
          'channel_index': channel_index,
          'eta': settings['eta'],
          'iterations': settings['iterations'],
          'max_loss': settings['max_loss'][i],
          'ann': settings['ann'],
          'lbfgs': settings['lbfgs'],
          'optim': settings['optim'],
      }
      dream = dream_and_save(np.copy(img), model, params, layer_dict, show=show, save=save, use_octaves=settings['use_octaves'])
      dreams.append(dream)

  return dreams


###MyDeepDream

In [0]:
def deep_dream(in_img, model, params, layer_dict=None):
  start = time.time()

  model_input = model.input
  layer_name = params['layer_name']
  channel_index = params['channel_index']
  eta = params['eta']
  iterations = params['iterations']
  max_loss = params['max_loss']
  lbfgs = params['lbfgs']
  optim = params['optim']
  if layer_dict == None:
    layer_dict = dict([(layer.name, layer) for layer in model.layers])
  
  if layer_name is not None:
    if layer_name not in layer_dict:
      raise ValueError('Layer ' + layer_name + ' not found in model.')
    layer = layer_dict[layer_name].output
    if layer.shape[3] < channel_index:
      raise ValueError('Layer ' + layer_name + ' has only {} channels.'.format(layer.shape[3]))
  
  ann = params['ann']

  inputs, targets = compute_input_output([in_img], [ann])
  regression_batch, labels_batch = targets

  img_batch = preprocess_image(np.copy(in_img))[np.newaxis, :, :, :]
  print(img_batch.dtype)

  # get layers
  classification_layer = layer_dict['classification']
  regression_layer = layer_dict['clipped_boxes']
  detections = layer_dict['filtered_detections']

  ##############################################################################
  #  loss and grads
  ##############################################################################

  # uncommentx this for layer or channel axtivations maximization and comment out retinanet loss
  # loss = K.mean(K.square(layer if channel_index is None else layer[:,:,:,channel_index]))


  # RetinaNet loss
  focal_loss = focal()
  smooth_L1_loss = smooth_l1()
  loss = focal_loss(labels_batch, classification_layer.output) + smooth_L1_loss(regression_batch, regression_layer.output)


  # gradients
  grads = K.gradients(loss, model_input)[0]

  fetch_loss_and_grads = K.function([model_input], [loss, grads, detections.output])

  d_images = []
  
  ##############################################################################
  #  LBFGS
  ##############################################################################
  if lbfgs:

    def fun(x):
      global lbfgs_iter
      x = np.reshape(x, img_batch.shape)
      outs = fetch_loss_and_grads([x.astype(np.float32)])
      print(lbfgs_iter, outs[0])
      lbfgs_iter += 1
      return outs[0].astype(np.float64), np.ravel(outs[1]).astype(np.float64)
    
    outs = fetch_loss_and_grads([img_batch])
    loss_value = outs[0]
    grad_values = outs[1]
    boxes = outs[2][0][0]
    scores = outs[2][1][0]
    labels = outs[2][2][0]
    detections = np.array(list(zip(scores, labels, boxes)))
    print(detections[:3])

    from scipy.optimize import minimize

    for i in range(iterations):
      res = minimize(fun, img_batch.astype(np.float64), method='L-BFGS-B', jac=True, options={'maxiter': 1000, 'disp': True, 'iprint':100} )
      print(res)
      img_batch = np.reshape(res.x.astype(np.float32), img_batch.shape)
      outs = fetch_loss_and_grads([img_batch])
      loss_value = outs[0]
      grad_values = outs[1]
      boxes = outs[2][0][0]
      scores = outs[2][1][0]
      labels = outs[2][2][0]
      detections = np.array(list(zip(scores, labels, boxes)))
      print(detections[:3])

      d_img = deprocess_image(img_batch[0])
      d_images.append((d_img, i))

      cv2_imshow(d_img)

      cv2_imshow(10 * np.abs(d_img.astype(np.float32) - in_img.astype(np.float32)).astype(np.uint8))

      img_batch = preprocess_image(d_img)[np.newaxis,:,:,:]

    process_time = time.time() - start
    print('Process time: {:.2f}'.format(process_time))

    d_images.append((deprocess_image(img_batch[0]),0))
    return img_batch, loss_value, process_time, d_images
  

  ##############################################################################
  #  gradient ascent/descent
  ##############################################################################
  
  for i in range(iterations):
    step_start = time.time()

    print()

    # fetch loss, grads and outs
    outs = fetch_loss_and_grads([img_batch])
    loss_value = outs[0]
    grad_values = outs[1]
    boxes = outs[2][0][0]
    scores = outs[2][1][0]
    labels = outs[2][2][0]

    old_img = deprocess_image(np.copy(img_batch[0]))
    
    print("Max grad val {}".format(np.max(np.abs(grad_values))))
    print(grad_values.shape)

    # update image wrt. gradients
    if eta is None:
      img_delta = 0.1 * grad_values / np.max(np.abs(grad_values)) 
    elif eta > 0:
      img_delta = eta * grad_values
    else:
      img_delta = np.sign(grad_values)
    
    img_batch += optim * img_delta       # optim = 1 for maximization, -1 for minimization

    # print out detections
    detections = np.array(list(zip(scores, labels, boxes)))
    print(detections[:3])

    if i % 10 == 0:
      d_img = deprocess_image(img_batch[0])
      d_images.append((d_img, i))
    
    # deprocess normalization
    if i % 100 == 0:
      d_img = deprocess_image(img_batch[0])
      cv2_imshow(d_img)

      print("Diff img")        
      print(np.max(np.abs(d_img.astype(np.float32) - in_img.astype(np.float32))))
      cv2_imshow((10 * np.abs(d_img.astype(np.float32) - in_img.astype(np.float32)).astype(np.uint8))[:,:,::-1])
      
      img_batch = preprocess_image(d_img)[np.newaxis,:,:,:]

    print("At step: {:3d}     loss:{:7.2f}     step_time:{:5.2f}".format(i, loss_value, time.time() - step_start))

    if max_loss is not None and loss_value > max_loss:
      break

  d_img = deprocess_image(img_batch[0])
  d_images.append((d_img, i))
  
  process_time = time.time() - start
  print('Process time: {:.2f}'.format(process_time))

  return img_batch, loss_value, process_time, d_images

###Min attack

In [0]:
def min_max_attack(in_img, model, params, layer_dict=None):
  start = time.time()

  model_input = model.input
  iterations = params['iterations']
  if layer_dict == None:
    layer_dict = dict([(layer.name, layer) for layer in model.layers])

  ann = params['ann']

  inputs, targets = compute_input_output([in_img], [ann])
  regression_batch, labels_batch = targets

  img_batch = preprocess_image(np.copy(in_img))[np.newaxis, :, :, :]

  # get layers  
  classification_layer = layer_dict['classification']
  regression_layer = layer_dict['clipped_boxes']
  detections = layer_dict['filtered_detections']
  
  # RetinaNet loss
  focal_loss = focal()
  smooth_L1_loss = smooth_l1()
  loss = focal_loss(labels_batch, classification_layer.output) + smooth_L1_loss(regression_batch, regression_layer.output)


  # gradients
  grads = K.gradients(loss, model_input)[0]

  fetch_loss_and_grads = K.function([model_input], [loss, grads, detections.output])

  for i in range(iterations):
    # fetch loss, grads and outs
    outs = fetch_loss_and_grads([img_batch])
    loss_value = outs[0]
    grad_values = outs[1]
    boxes = outs[2][0][0]
    scores = outs[2][1][0]
    labels = outs[2][2][0]
    
    print("Max grad val {}".format(np.max(np.abs(grad_values))))
    print(grad_values.shape)

    n = params['px']
    rgrads = np.abs(np.ravel(grad_values))
    arm = np.argsort(rgrads)
    for j in range(n):
      idx = np.unravel_index(arm[-j], grad_values.shape)
      # print('idx', j, idx)
      img_batch[idx] -= 255 * np.sign(grad_values[idx])
      if j % params['print'] == 0:
        dtnow = datetime.now()
        timestamp = dtnow.strftime("%Y-%m-%d_%H-%M-%S")
        savename = params['save_path'] + timestamp + '_' + str(n) + '_' + str(j)
        d_img = deprocess_image(img_batch[0])
        plt.figure(figsize=(10,10))
        diff = (10 * np.abs(d_img.astype(np.float32) - in_img.astype(np.float32)).astype(np.uint8))
        plt.imshow(diff[:,:,::-1])
        plt.show()      
        cv2.imwrite('{}-diff.png'.format(savename), diff)
        cv2.imwrite('{}-clean.png'.format(savename), d_img)
        detects = detect(d_img, save=savename)

    d_img = deprocess_image(img_batch[0])

    plt.figure(figsize=(10,10))
    plt.imshow((10 * np.abs(d_img.astype(np.float32) - in_img.astype(np.float32)).astype(np.uint8))[:,:,::-1])
    plt.show()

  return img_batch, loss_value

###Adversarial

In [0]:
def adversarial(in_img, model, params, layer_dict=None):
  model_input = model.input
  iterations = params['iterations']
  optim = params['optim']
  if layer_dict == None:
    layer_dict = dict([(layer.name, layer) for layer in model.layers])
  
  ann = params['ann']

  inputs, targets = compute_input_output([in_img], [ann])
  regression_batch, labels_batch = targets

  img_batch = preprocess_image(np.copy(in_img))[np.newaxis, :, :, :]

  # get layers
  classification_layer = layer_dict['classification']
  regression_layer = layer_dict['clipped_boxes']
  detections = layer_dict['filtered_detections']

  ##############################################################################
  #  loss and grads
  ##############################################################################

  # RetinaNet loss
  focal_loss = focal()
  smooth_L1_loss = smooth_l1()
  loss = focal_loss(labels_batch, classification_layer.output) + smooth_L1_loss(regression_batch, regression_layer.output)

  # gradients
  grads = K.gradients(loss, model_input)[0]

  fetch_loss_and_grads = K.function([model_input], [loss, grads, detections.output])

  d_images = []
  
  ##############################################################################
  #  gradient ascent/descent
  ##############################################################################
  
  for i in range(iterations):
    print()

    # fetch loss, grads and outs
    outs = fetch_loss_and_grads([img_batch])
    loss_value = outs[0]
    grad_values = outs[1]
    boxes = outs[2][0][0]
    scores = outs[2][1][0]
    labels = outs[2][2][0]

    old_img = deprocess_image(np.copy(img_batch[0]))
    
    print("Max grad val {}".format(np.max(np.abs(grad_values))))
    print(grad_values.shape)

    # update image wrt. gradients
    img_delta = np.sign(grad_values)
    img_batch += optim * img_delta       # optim = 1 for maximization, -1 for minimization
    
    # print out detections
    detections = np.array(list(zip(scores, labels, boxes)))
    print(detections[:3])

    if i % params['print'] == 0:

      timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
      savename = params['save_path'] + timestamp + '_adv_' + str(i)
      d_img = deprocess_image(img_batch[0])
      d_images.append((d_img, i))

      diff = (10 * np.abs(d_img.astype(np.float32) - in_img.astype(np.float32)).astype(np.uint8))
      plt.figure(figsize=(10,10))
      plt.imshow(diff[:,:,::-1])
      plt.show()      
      cv2.imwrite('{}-diff.png'.format(savename), diff)
      detects = detect(d_img, save=savename)

    # deprocess normalization
    if i % 100 == 0:
      plt.imshow(d_img[:,:,::-1])     #### RGB len pre zobrazenie
      plt.show()

      print("Diff img")         
      print(np.max(np.abs(d_img.astype(np.float32) - in_img.astype(np.float32))))

      plt.imshow((10 * np.abs(d_img.astype(np.float32) - in_img.astype(np.float32)).astype(np.uint8))[:,:,::-1])
      plt.show()
      
      img_batch = preprocess_image(d_img)[np.newaxis,:,:,:]

    print("At step: {:3d}     loss:{:7.2f}".format(i, loss_value))

  d_img = deprocess_image(img_batch[0])
  d_images.append((d_img, i))
  

  return img_batch, loss_value, d_images

###DeepDream attack

In [0]:
def deep_dream_attack(in_img, model, params, layer_dict=None):
  model_input = model.input
  eta = params['eta']
  iterations = params['iterations']
  lbfgs = params['lbfgs']
  optim = params['optim']
  if layer_dict == None:
    layer_dict = dict([(layer.name, layer) for layer in model.layers])
  
  ann = params['ann']

  inputs, targets = compute_input_output([in_img], [ann])
  regression_batch, labels_batch = targets

  img_batch = preprocess_image(np.copy(in_img))[np.newaxis, :, :, :]

  # get layers
  classification_layer = layer_dict['classification']
  regression_layer = layer_dict['clipped_boxes']
  detections = layer_dict['filtered_detections']

  ##############################################################################
  #  loss and grads
  ##############################################################################

  # RetinaNet loss
  focal_loss = focal()
  smooth_L1_loss = smooth_l1()
  loss = focal_loss(labels_batch, classification_layer.output) + smooth_L1_loss(regression_batch, regression_layer.output)

  # gradients
  grads = K.gradients(loss, model_input)[0]

  fetch_loss_and_grads = K.function([model_input], [loss, grads, detections.output])

  d_images = []
  
  ##############################################################################
  #  LBFGS
  ##############################################################################
  if lbfgs:

    def fun(x):
      global lbfgs_iter
      x = np.reshape(x, img_batch.shape)
      outs = fetch_loss_and_grads([x.astype(np.float32)])
      print(lbfgs_iter, outs[0])
      lbfgs_iter += 1
      return outs[0].astype(np.float64), np.ravel(outs[1]).astype(np.float64)
    
    outs = fetch_loss_and_grads([img_batch])
    loss_value = outs[0]
    grad_values = outs[1]
    boxes = outs[2][0][0]
    scores = outs[2][1][0]
    labels = outs[2][2][0]
    detections = np.array(list(zip(scores, labels, boxes)))
    print(detections[:3])

    from scipy.optimize import minimize

    for i in range(iterations):
      res = minimize(fun, img_batch.astype(np.float64), method='L-BFGS-B', jac=True, options={'maxiter': 1000, 'disp': True, 'iprint':100} )
      print(res)
      img_batch = np.reshape(res.x.astype(np.float32), img_batch.shape)
      outs = fetch_loss_and_grads([img_batch])
      loss_value = outs[0]
      grad_values = outs[1]
      boxes = outs[2][0][0]
      scores = outs[2][1][0]
      labels = outs[2][2][0]
      detections = np.array(list(zip(scores, labels, boxes)))
      print(detections[:3])

      d_img = deprocess_image(img_batch[0])
      d_images.append((d_img, i))

      plt.imshow(d_img[:,:,::-1])     #### RGB len pre zobrazenie
      plt.show()

      plt.imshow(10 * np.abs(d_img.astype(np.float32) - in_img.astype(np.float32)).astype(np.uint8))
      plt.show()

      img_batch = preprocess_image(d_img)[np.newaxis,:,:,:]

    d_images.append((deprocess_image(img_batch[0]),0))
    return img_batch, loss_value, d_images
  

  ##############################################################################
  #  gradient ascent/descent
  ##############################################################################
  
  for i in range(iterations):
    print()

    # fetch loss, grads and outs
    outs = fetch_loss_and_grads([img_batch])
    loss_value = outs[0]
    grad_values = outs[1]
    boxes = outs[2][0][0]
    scores = outs[2][1][0]
    labels = outs[2][2][0]    

    # update image wrt. gradients
    if eta is None:
      img_delta = 0.1 * grad_values / np.max(np.abs(grad_values)) 
    elif eta > 0:
      img_delta = eta * grad_values
    else:
      img_delta = np.sign(grad_values)
    
    img_batch += optim * img_delta       # optim = 1 for maximization, -1 for minimization
    
    # print out detections
    detections = np.array(list(zip(scores, labels, boxes)))
    print(detections[:3])

    if i % params['print'] == 0:
      timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
      savename = params['save_path'] + timestamp + '_dda_' + str(i)
      d_img = deprocess_image(img_batch[0])
      d_images.append((d_img, i))

      cv2_imshow(d_img)
      cv2.imwrite('{}-d_img.png'.format(savename), d_img)

      diff = (10 * np.abs(d_img.astype(np.float32) - in_img.astype(np.float32)).astype(np.uint8))

      cv2_imshow(diff)     
      cv2.imwrite('{}-diff.png'.format(savename), diff)
      detects = detect(d_img, save=savename)

    
    # deprocess normalization
    if i % params['dep_norm'] == 0:
      d_img = deprocess_image(img_batch[0])

      img_batch = preprocess_image(d_img)[np.newaxis,:,:,:]
    
    print("At step: {:3d}     loss:{:7.2f}".format(i, loss_value))

  d_img = deprocess_image(img_batch[0])
  d_images.append((d_img, i))
  
  return d_img, loss_value, d_images

###Settings – DeepDream

In [0]:
lbfgs_iter = 0 # ugly method how to show iter number in lbfgs
settings = {
  'layers': [None],     # put 'P3', 'P4', ... , 'P7' to analyse layer
  'max_loss': [None],   # for each layer sepparately
  'ch_indexes': [None], # choose channels or leave None for analisis of the whole layer
  'eta': 1000,          # None for automatic eta / 0 for sign gradient optimization / >0 for constant eta
  'iterations': 100,
  'use_octaves': False,
  'lbfgs': False,
  'optim': -1,          # 1 for maximization / -1 for minimization

  'ann': {              # at least one annotation shold be there, even if it is not used
    'labels': np.array([14]), # bird
    'bboxes': np.array([
                    np.array([250.0, 40.0, 350.0, 110.0])
              ])
    },
  # 'ann': {
  #   'labels': np.array([0]), # person
  #   'bboxes': np.array([
  #                   np.array([180.0, 150.0, 320.0, 480.0])
  #             ])
  #   },
  # 'ann': {
  #   'labels': np.array([4]), # plane
  #   'bboxes': np.array([
  #                   np.array([50.0, 80.0, 150.0, 150.0])
  #             ])
  #   },
  # 'ann': {
  #   'labels': np.array([20]), # elephant
  #   'bboxes': np.array([
  #                   np.array([100.0, 200.0, 400.0, 450.0])
  #             ])
  #   },
  # 'ann': {
  #   'labels': np.array([47]), # aplle
  #   'bboxes': np.array([
  #                   np.array([80.0, 360.0, 150.0, 430.0])
  #             ])
  #   },
}


path = drive_path + 'images/dogs.jpg'  # replace by your desired images
img = read_image_bgr(path)

# seda
# img = 127 * np.ones([500,500,3], dtype=np.uint8)

# random uniform noise
# img = np.random.uniform(size=[500,500,3], low=0.0, high=255.0).astype(np.uint8)

# cierna
#img = np.zeros([600,600,3], dtype=np.uint8)

# biela 
#img = 255 * np.ones([600,600,3], dtype=np.uint8)


# measure total process time
start_time = time.time()

# run deepdream
dreams = run_dream(img, model, settings, show=True, save=True)
# boxes, scores, labels = detect(dreams[0][0], save=dreams[0][2], min_score=0.5, anns=settings['ann']) # uncomment to run detection

total_process_time = time.time() - start_time
print('TOTAL process time: {:.2f}'.format(total_process_time))

###Settings2 – DeepDream

In [0]:
# just another cells for deepdreaming

path = drive_path + 'images/dog.jpg'
img = read_image_bgr(path)

boxes, scores, labels = detect(img)

# synthetized annotations
_labels, _bboxes = [], []
for box, score, label in zip(boxes, scores, labels):
  if score < 0.5:
    break
  _labels.append(label)
  _bboxes.append(box)

digital_anns = {
  'labels': np.array(_labels),
  'bboxes': np.array(_bboxes)
}
digital_anns

In [0]:
print(digital_anns)
lbfgs_iter = 0 # ugly way how to show iter number in lbfgs
settings2 = {
  'layers': [None],
  'max_loss': [None], 
  'ch_indexes': [None],
  'eta': None,  # None for automatic eta / 0 for sign gradient optimization / >0 for constant eta
  'iterations': 1000,
  'use_octaves': False,
  'ann': digital_anns,
  'lbfgs': False,
  'optim': 1,
}

dreams = run_dream(np.copy(img), model, settings2, show=True, save=True)
boxes, scores, labels = detect(dreams[0][0], save=dreams[0][2], min_score=0.5)
print(np.array(list(zip(scores, labels, boxes)))[:10])

###Settings3 – Minimal attack

In [0]:
path = drive_path + 'images/stop.jpg'
img = read_image_bgr(path)

boxes, scores, labels = detect(img)

# synthetized annotations
_labels, _bboxes = [], []
for box, score, label in zip(boxes, scores, labels):
  if score < 0.5:
    break
  _labels.append(label)
  _bboxes.append(box)

digital_anns = {
  'labels': np.array(_labels),
  'bboxes': np.array(_bboxes)
}
digital_anns

In [0]:
settings3 = {
  'iterations': 1,
  'px': 100000,
  'ann': digital_anns,
  'save_path': save_path,
  'print': 10000,
}

dtnow = datetime.now()
timestamp = dtnow.strftime("%Y-%m-%d_%H-%M-%S")
savename = save_path + timestamp + str(settings3['px']) + str(digital_anns)

attack = min_max_attack(np.copy(img), model, settings3)
img_res = deprocess_image(attack[0][0])
dets = detect(img_res, save=savename)

###Settings4 – Adversarial attack

In [0]:
path = drive_path + 'images/stop.jpg'
img = read_image_bgr(path)

boxes, scores, labels = detect(img)

_labels, _bboxes = [], []

for box, score, label in zip(boxes, scores, labels):
  if score < 0.5:
    break
  if label == 0:
    _labels.append(label)
    _bboxes.append(box)

# _labels, _bboxes = [], [] # uncomment this if we want empty annotations

digital_anns = {
  'labels': np.array(_labels),
  'bboxes': np.array(_bboxes)
}
digital_anns

In [0]:
settings4 = {
  'iterations': 5,
  'optim': -1,
  'ann': digital_anns,
  'save_path': save_path,
  'print': 1,
}

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
savename = save_path + timestamp + str(settings4['iterations'])

attack = adversarial(np.copy(img), model, settings4)
img_res = deprocess_image(attack[0][0])
dets = detect(img_res, save=savename)

###Settings5 – DeepDream attack

In [0]:
path = drive_path + 'images/stop.jpg'
img = read_image_bgr(path)

boxes, scores, labels = detect(img)
_labels, _bboxes = [], []
for box, score, label in zip(boxes, scores, labels):
  if score < 0.5:
    break
  _labels.append(label)
  _bboxes.append(box)

# _labels, _bboxes = [], [] # uncomment this if we want empty annotations

digital_anns = {
  'labels': np.array(_labels),
  'bboxes': np.array(_bboxes)
}
digital_anns

In [0]:
lbfgs_iter = 0 # ugly method how to show iter number in lbfgs

settings5 = { 
  'layers': [None],
  'max_loss': [None],
  'ch_indexes': [None], 
  'eta': 1000, # None for automatic eta / 0 for sign gradient optimization / >0 for constant eta
  'iterations': 10,
  'use_octaves': False,
  'lbfgs': False,
  'optim': -1,
  'dep_norm': 100,
  'print': 1,
  'save_path': save_path,
  'ann': digital_anns,

}

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
savename = save_path + timestamp + str(settings5['iterations'])

# run deepdream
dreams = deep_dream_attack(img, model, settings5)    # 1.[] - dream, 2.[0] - result image / [2] - savepath 
boxes, scores, labels = detect(dreams[0], save=savename, min_score=0.5, anns=None)
print(np.array(list(zip(scores, labels, boxes)))[:10])